In [3]:
### Required Libraries ###
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Functionality Helper Functions ###
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")


In [4]:
def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }


In [5]:
### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]


def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }


In [6]:
def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response


In [7]:
### Intents Handlers ###
def recommend_portfolio(intent_request):
    """
    Performs dialog management and fulfillment for recommending a portfolio.
    """

    first_name = get_slots(intent_request)["firstName"]
    age = get_slots(intent_request)["age"]
    investment_amount = get_slots(intent_request)["investmentAmount"]
    risk_level = get_slots(intent_request)["riskLevel"]
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.

        def validate_info(age, investment_amount):
            if age is not None:
                birth_date = datetime.strptime(age, "%Y-%m-%d")
                age_result = relativedelta(datetime.now(), birth_date).years
                if age_result < 21:
                    return build_validation_result(
                        False, 
                        "age"
                        "You must be at least 21 years of age to use this program"
                        "Please use another date of birth", 
                        )
                        
            if investment_amount is not None:
                investment_amount = parse_float(investment_amount)
                if investment_amount < 5000:
                    return build_validation_result(
                        False,
                        "investment_amount",
                        "The minimum amount to invest is $5,000. " 
                        "Please provide an amount of at least $5,000.",
                        )
            return build_validation_result(True, None, None)
        
                
        ### YOUR DATA VALIDATION CODE ENDS HERE ###


In [8]:
  # Fetch current session attibutes
def get_slots(intent_request):
    output_session_attributes = intent_request["sessionAttributes"]

    return delegate(output_session_attributes, get_slots(intent_request))


In [9]:
 # Get the initial investment recommendation
def get_intent_request(risk_level):
    initial_recommendation = get_intent_request(risk_level)

  

In [12]:
# Return a message with the initial recommendation based on the risk level.
def get_intent_request(risk_level):
    return close(
    intent_request["sessionAttributes"], "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """{} thank you for your information;
            based on the risk level you defined, my recommendation is to choose an investment portfolio with {}
            """.format(
                first_name, initial_recommendation), }, )


In [13]:
### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "RecommendPortfolio":
        return recommend_portfolio(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


In [14]:
### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)
